In [167]:
# Lab 5 Logistic Regression Classifier
import tensorflow as tf
import numpy as np
import math

tf.set_random_seed(777)  # for reproducibility

# x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
# y_data = [[0], [0], [0], [1], [1], [1]]


In [168]:
import pandas as pd
dataframe = pd.read_csv("titanic-train.csv")
# x_data_raw = [i[2:] for i in data[:].values] # need to remove names, change gender to binary, deal with tickets, cabin and embarked
# # x_data = [[i] for j in x_data_raw for i in j if (type(i) !=type("hi") or type(i)!=type(nan))]
# for i in range(len(x_data_raw)):
#     x_data_raw[i] = [k for k in x_data_raw[i] if (type(k) !=type("hi"))]
#     x_data_raw[i] = x_data_raw[i][:-1]
# y_data = [[i] for i in data["Survived"].values]
# print(data.keys())
# print(x_data_raw[:3])
# print(y_data[:3])

In [169]:
dataframe = dataframe.drop(["PassengerId", "Name", "Ticket"], axis=1)
dataframe["Cabin"] = dataframe["Cabin"].apply(lambda data: 0 if pd.isnull(data) else 1)
dataframe["Sex"] = dataframe["Sex"].apply(lambda gender: 0 if gender == "male" else 1)
dataframe["Embarked"] = dataframe["Embarked"].apply(lambda data: 0 if data == "S" else 1 if data == "Q" else 2)
dataframe["Age"] = dataframe["Age"].apply(lambda age: 0 if pd.isnull(age) else age)
y_data = np.array([[i] for i in dataframe["Survived"].values])
x_data = np.array(dataframe.drop("Survived", axis=1).values)

test_dataframe = pd.read_csv("titanic-test.csv")
test_dataframe = test_dataframe.drop(["PassengerId", "Name", "Ticket"], axis=1)
test_dataframe["Cabin"] = test_dataframe["Cabin"].apply(lambda data: 0 if pd.isnull(data) else 1)
test_dataframe["Sex"] = test_dataframe["Sex"].apply(lambda gender: 0 if gender == "male" else 1)
test_dataframe["Embarked"] = test_dataframe["Embarked"].apply(lambda data: 0 if data == "S" else 1 if data == "Q" else 2)
test_dataframe["Age"] = test_dataframe["Age"].apply(lambda age: 0 if pd.isnull(age) else age)
x_test_data = np.array(test_dataframe.values)


# Notes
#### Sigmoid function
f(z) = 1/(1+e^-z)
'S' like function
Squeezes output between 0 and 1

#### ylog
Mean square error does not always work nicely, so people came up with an alternative
if y is close to 1

In [170]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

w1 = tf.Variable(tf.truncated_normal([8, 64], stddev=1 / math.sqrt(8)), name='weight') # [2,1] because of shape of X and Y
b1 = tf.Variable(tf.zeros([64]), name='bias')

w2 = tf.Variable(tf.truncated_normal([64, 1], stddev=1 / math.sqrt(64)), name='weight') # [2,1] because of shape of X and Y
b2 = tf.Variable(tf.zeros([1]), name='bias')


In [171]:
# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
o1 = tf.nn.elu(tf.matmul(X, w1) + b1)
hypothesis = tf.sigmoid(tf.matmul(o1, w2) + b2) # activation functiion

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis + 1e-15) + (1 - Y) *
                       tf.log(1 - hypothesis + 1e-15))

train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [172]:
# Accuracy computation
# True if hypothesis>0.5 else False
#accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1)), dtype=tf.float32))


In [180]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
        if step % 500 == 0:
            print(step, cost_val)

    # Accuracy report
    h = sess.run(hypothesis,
                       feed_dict={X: x_test_data})
    print(tf.round(h).eval())

0 1.83996
500 0.334309
1000 0.297143
1500 0.255449
2000 0.239503
2500 0.225591
3000 0.219235
3500 0.207692
4000 0.197052
4500 0.189266
5000 0.18192
5500 0.236901
6000 0.173025
6500 0.176991
7000 0.166614
7500 0.165835
8000 0.161671
8500 0.159246
9000 0.156656
9500 0.154322
10000 0.15193


TypeError: 'j' is an invalid keyword argument for this function

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`